# Introduction

This notebook explores getting data via the SEC's frames API documented here: https://www.sec.gov/edgar/sec-api-documentation.

"The xbrl/frames API aggregates one fact for each reporting entity that is last filed that most closely fits the calendrical period requested. This API supports for annual, quarterly and instantaneous data:

https://data.sec.gov/api/xbrl/frames/us-gaap/AccountsPayableCurrent/USD/CY2019Q1I.json
Where the units of measure specified in the XBRL contains a numerator and a denominator, these are separated by “-per-” such as “USD-per-shares”. Note that the default unit in XBRL is “pure”.

The period format is CY#### for annual data (duration 365 days +/- 30 days), CY####Q# for quarterly data (duration 91 days +/- 30 days), and CY####Q#I for instantaneous data. Because company financial calendars can start and end on any month or day and even change in length from quarter to quarter to according to the day of the week, the frame data is assembled by the dates that best align with a calendar quarter or year. Data users should be mindful different reporting start and end dates for facts contained in a frame."

# Imports

In [1]:
import requests 
import pandas as pd
from src.data import get_ticker_cik_map, sample_company_schemas, get_all_concepts
from src.visualize import plot_missing_data

# Schema Exploration

Which concepts do companies report on? Which are the most commonly reported?

In [2]:
ticker_cik_map = get_ticker_cik_map()
ticker_cik_map.head()

,ticker,cik
0,aapl,0000320193
1,msft,0000789019
2,brk-b,0001067983
3,unh,0000731766
4,jnj,0000200406


In [3]:
headers = headers = {
    'User-Agent': 'Andrew Abeles andrewabeles@sandiego.edu'
}

In [4]:
# analyze the schemas of 100 random companies 
sample_schema = sample_company_schemas(headers, ticker_cik_map, n=100, random_state=1)
sample_schema.shape

(5254, 5)

In [5]:
sample_schema.head()

,taxonomy,tag,unit,description,companies_reported
0,us-gaap,Assets,USD,Sum of the carrying amounts as of the balance ...,81
1,us-gaap,LiabilitiesAndStockholdersEquity,USD,"Amount of liabilities and equity items, includ...",81
2,us-gaap,NetCashProvidedByUsedInFinancingActivities,USD,Amount of cash inflow (outflow) from financing...,81
3,us-gaap,NetIncomeLoss,USD,"The portion of profit or loss for the period, ...",81
4,us-gaap,NetCashProvidedByUsedInOperatingActivities,USD,Amount of cash inflow (outflow) from operating...,81


In [6]:
# concepts reported by more than 50 of the 100 companies sampled 
common_concepts = sample_schema.query("companies_reported > 50")
common_concepts

,taxonomy,tag,unit,description,companies_reported
0,us-gaap,Assets,USD,Sum of the carrying amounts as of the balance ...,81
1,us-gaap,LiabilitiesAndStockholdersEquity,USD,"Amount of liabilities and equity items, includ...",81
2,us-gaap,NetCashProvidedByUsedInFinancingActivities,USD,Amount of cash inflow (outflow) from financing...,81
3,us-gaap,NetIncomeLoss,USD,"The portion of profit or loss for the period, ...",81
4,us-gaap,NetCashProvidedByUsedInOperatingActivities,USD,Amount of cash inflow (outflow) from operating...,81
5,us-gaap,StockholdersEquity,USD,Total of all stockholders' equity (deficit) it...,79
6,us-gaap,NetCashProvidedByUsedInInvestingActivities,USD,Amount of cash inflow (outflow) from investing...,79
7,us-gaap,RetainedEarningsAccumulatedDeficit,USD,The cumulative amount of the reporting entity'...,79
8,us-gaap,CashAndCashEquivalentsAtCarryingValue,USD,Amount of currency on hand as well as demand d...,76
9,dei,EntityPublicFloat,USD,The aggregate market value of the voting and n...,72


# Data Retrieval

Now let's get all the reporting companies' data on these common concepts from the frames API.

In [7]:
period = 'CY2020Q4I' # most recent and complete fiscal year frame

In [8]:
df = get_all_concepts(headers, period, common_concepts)
df.shape

(7645, 42)

In [9]:
df.head()

concept,AccountsPayableCurrent_USD,AccumulatedOtherComprehensiveIncomeLossNetOfTax_USD,AdditionalPaidInCapital_USD,AssetsCurrent_USD,Assets_USD,CashAndCashEquivalentsAtCarryingValue_USD,CashAndCashEquivalentsPeriodIncreaseDecrease_USD,CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseIncludingExchangeRateEffect_USD,CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalents_USD,CommonStockSharesAuthorized_shares,...,ProceedsFromIssuanceOfCommonStock_USD,ProfitLoss_USD,PropertyPlantAndEquipmentNet_USD,RetainedEarningsAccumulatedDeficit_USD,ShareBasedCompensation_USD,StockIssuedDuringPeriodValueNewIssues_USD,StockholdersEquity_USD,UnrecognizedTaxBenefits_USD,WeightedAverageNumberOfDilutedSharesOutstanding_shares,WeightedAverageNumberOfSharesOutstandingBasic_shares
cik,,,,,,,,,,,,,,,,,,,,,
1750,198300000.0,-4.300000e+07,NaN,1.041700e+09,1.669300e+09,1.100000e+08,NaN,NaN,NaN,1.000000e+08,...,NaN,NaN,1.250000e+08,6.996000e+08,NaN,NaN,NaN,NaN,NaN,NaN
1800,NaN,-8.946000e+09,NaN,2.044100e+10,7.254800e+10,6.838000e+09,NaN,NaN,6.838000e+09,2.400000e+09,...,NaN,NaN,9.029000e+09,2.762700e+10,NaN,NaN,3.278400e+10,1.210000e+09,NaN,NaN
1961,981898.0,NaN,41240880.0,4.745870e+05,6.918540e+05,4.745870e+05,NaN,NaN,NaN,2.500000e+08,...,NaN,NaN,NaN,4.517450e+07,NaN,NaN,-2.669888e+06,NaN,NaN,NaN
2098,7601123.0,-8.259670e+05,7930673.0,8.368591e+07,1.298677e+08,4.167376e+06,NaN,NaN,4.167000e+06,8.000000e+06,...,NaN,NaN,2.023891e+07,5.803325e+07,NaN,NaN,6.271700e+07,NaN,NaN,NaN
2178,85991000.0,NaN,13340000.0,1.875130e+08,2.961870e+08,3.929300e+07,NaN,NaN,5.206500e+07,7.500000e+06,...,NaN,NaN,9.413400e+07,1.353290e+08,NaN,NaN,1.490920e+08,NaN,NaN,NaN


In [10]:
# move CIK from the index to a column
df.reset_index(inplace=True)
df.head()

concept,cik,AccountsPayableCurrent_USD,AccumulatedOtherComprehensiveIncomeLossNetOfTax_USD,AdditionalPaidInCapital_USD,AssetsCurrent_USD,Assets_USD,CashAndCashEquivalentsAtCarryingValue_USD,CashAndCashEquivalentsPeriodIncreaseDecrease_USD,CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseIncludingExchangeRateEffect_USD,CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalents_USD,...,ProceedsFromIssuanceOfCommonStock_USD,ProfitLoss_USD,PropertyPlantAndEquipmentNet_USD,RetainedEarningsAccumulatedDeficit_USD,ShareBasedCompensation_USD,StockIssuedDuringPeriodValueNewIssues_USD,StockholdersEquity_USD,UnrecognizedTaxBenefits_USD,WeightedAverageNumberOfDilutedSharesOutstanding_shares,WeightedAverageNumberOfSharesOutstandingBasic_shares
0,1750,198300000.0,-4.300000e+07,NaN,1.041700e+09,1.669300e+09,1.100000e+08,NaN,NaN,NaN,...,NaN,NaN,1.250000e+08,6.996000e+08,NaN,NaN,NaN,NaN,NaN,NaN
1,1800,NaN,-8.946000e+09,NaN,2.044100e+10,7.254800e+10,6.838000e+09,NaN,NaN,6.838000e+09,...,NaN,NaN,9.029000e+09,2.762700e+10,NaN,NaN,3.278400e+10,1.210000e+09,NaN,NaN
2,1961,981898.0,NaN,41240880.0,4.745870e+05,6.918540e+05,4.745870e+05,NaN,NaN,NaN,...,NaN,NaN,NaN,4.517450e+07,NaN,NaN,-2.669888e+06,NaN,NaN,NaN
3,2098,7601123.0,-8.259670e+05,7930673.0,8.368591e+07,1.298677e+08,4.167376e+06,NaN,NaN,4.167000e+06,...,NaN,NaN,2.023891e+07,5.803325e+07,NaN,NaN,6.271700e+07,NaN,NaN,NaN
4,2178,85991000.0,NaN,13340000.0,1.875130e+08,2.961870e+08,3.929300e+07,NaN,NaN,5.206500e+07,...,NaN,NaN,9.413400e+07,1.353290e+08,NaN,NaN,1.490920e+08,NaN,NaN,NaN


In [11]:
# write the data to a file 
df.to_csv('../data/raw/companies.csv', index=False)